In [1]:


from ema_workbench import (Model, RealParameter, ScalarOutcome,
                           MultiprocessingEvaluator, ema_logging,
                           Constant, Scenario, Constraint,optimize)


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [22]:
sufficientarian_threshold=[100,50]
supply_percapita_outcomes={"a":120,"b":1, "c":99}
population = {"a":100,"b":500, "c":10}
names = ["a", "b", "c"]

sumprod= np.sum([supply_percapita_outcomes[ZA] * population[ZA] for ZA in names])/np.sum(list(population.values()))




print(sumprod)


22.114754098360656


In [77]:
input_data_dict = pd.read_csv("data/input_data.csv").set_index('Variable').to_dict()['Value']
input_data_dict

{'service_intakes_PP1': 48064,
 'service_intakes_PP2': 8051,
 'service_intakes_PP3': 14887,
 'service_intakes_Toluquilla': 9412,
 'service_intakes_Pozos': 11910,
 'industry_intakes_PP1': 2208,
 'industry_intakes_PP2': 281,
 'industry_intakes_PP3': 385,
 'industry_intakes_Toluquilla': 210,
 'industry_intakes_Pozos': 263,
 'public_intakes_PP1': 2286,
 'public_intakes_PP2': 426,
 'public_intakes_PP3': 747,
 'public_intakes_Toluquilla': 454,
 'public_intakes_Pozos': 842,
 'domestic_intakes_PP1': 445436,
 'domestic_intakes_PP2': 125446,
 'domestic_intakes_PP3': 147032,
 'domestic_intakes_Toluquilla': 128862,
 'domestic_intakes_Pozos': 199613,
 'domestic_consumption': 128,
 'service_consumption': 558,
 'industry_consumption': 4457,
 'public_consumption': 6272}

In [78]:
locals().update(input_data_dict)

In [2]:
from AMG_function import AMG_model_function
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import ema_workbench
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, ema_logging)

#Setting the stage

info_indicator = "supplied"
justice_indicator_to_minimize = 'supplied_demand_GINI'
experiment_name = "space_filling"

ema_logging.log_to_stderr(ema_logging.INFO)

ZA_names = ["PP1", "PP2", "PP3", "Toluquilla", "Pozos"]

#instantiate the model
AMG_model = Model("AMGmodel", function = AMG_model_function)

#Levers

AMG_model.levers = []

low_flow =0
high_flow= 1.5
maximum_flows = {"chapala": 7.6, "calderon":1.3, "zapotillo":1, "pozos":2.5, "toluquilla":1} #modify zapotillo and calderon to one system to handle max better and simplify?
AMG_model.uncertainties = [RealParameter('chapala_flow',low_flow,maximum_flows["chapala"]),
                           RealParameter('calderon_flow',low_flow,maximum_flows["calderon"]),
                           RealParameter('zapotillo_flow',low_flow,maximum_flows["zapotillo"]),
                           RealParameter('pozos_flow',low_flow,maximum_flows["pozos"]),
                           RealParameter('toluquilla_flow',low_flow,maximum_flows["toluquilla"])]

#Outcomes
if info_indicator != "":
    info_outcomes = [f"{info_indicator}_{ZA}" for ZA in ZA_names]
else: info_outcomes = []


AMG_model.outcomes = [ScalarOutcome(info_outcome, kind=ScalarOutcome.INFO) for info_outcome in info_outcomes]


c:\Users\ariel\anaconda3\envs\MBWSA\lib\site-packages\ema_workbench\em_framework\evaluators.py:58: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


In [6]:
ema_logging.log_to_stderr(ema_logging.INFO)

n_scenarios = 1

with MultiprocessingEvaluator(AMG_model, n_processes=-1) as evaluator:
    np.random.seed(1)
    scenarios_data = evaluator.perform_experiments(n_scenarios)

import seaborn as sns

df = scenarios_data[0]
df_plot = df.drop(columns=["model","policy"])
experiment_name = "spacefilling 0-2.5"

df.head()

[MainProcess/INFO] pool started with 7 workers
[MainProcess/INFO] performing 1 scenarios * 1 policies * 1 model(s) = 1 experiments
100%|████████████████████████████████████████████| 1/1 [00:04<00:00,  4.99s/it]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


,calderon_flow,chapala_flow,pozos_flow,toluquilla_flow,zapotillo_flow,scenario,policy,model
0,0.542129,5.474466,0.000286,0.302333,0.146756,3,None,AMGmodel
